In [340]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats

In [341]:
file = '/content/drive/MyDrive/datasets/Ex - Salarios.csv'
df = pd.read_csv(file)
df

,anos de estudo,anos de experiencia,idade,salario
0,18.0,6.0,35.0,"16440,58"
1,15.0,12.0,39.0,"15992,51"
2,18.0,11.0,43.0,"15009,94"
3,2.0,2.0,14.0,"6964,13"
4,NaN,6.0,18.0,"7874,42"
5,NaN,16.0,40.0,"18329,56"
6,NaN,4.0,34.0,"15677,25"
7,NaN,28.0,42.0,"9429,71"
8,NaN,27.0,53.0,"15743,70"
9,NaN,12.0,29.0,"15922,81"


In [342]:
df.isnull().sum()

anos de estudo         7
anos de experiencia    8
idade                  7
salario                0
dtype: int64

In [343]:
df.columns = df.columns.str.replace(' ', '_')
df['salario'] = df['salario'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

In [344]:
df1 = df
df_filtered = df1.query('anos_de_estudo.notnull() and anos_de_experiencia.notnull() and idade.notnull()')

In [345]:
x = df_filtered[['anos_de_estudo', 'anos_de_experiencia', 'idade']]
y = df_filtered['salario'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score_original = model.score(x, y)
print(score_original)

0.9014024420114483


In [346]:
# Reconstruir idade
df_filtered = df1[ (df1['anos_de_estudo'].notnull()) & (df1['anos_de_experiencia'].notnull()) & (df1['idade'].notnull()) ]
x = df_filtered[['anos_de_estudo', 'anos_de_experiencia']].to_numpy()
y = df_filtered['idade'].to_numpy()
model_idade = LinearRegression()
model_idade.fit(x, y)
score = model_idade.score(x, y)
score

0.9892414636435288

In [347]:
# minha função criada para completar o ano de cada linha
def completar_idade(linha):
  if pd.isna(linha['idade']) and not pd.isna(linha['anos_de_estudo']) and not pd.isna(linha['anos_de_experiencia']):
    # Separa as variáveis regressoras utilizadas no modelo
    campos = linha[['anos_de_estudo','anos_de_experiencia']]
    # Retorna para o "apply" o valor previsto
    return round(model_idade.predict([campos])[0])
  return linha['idade']

# apply: executar a função chamada 1x para cada linha
df1['idade_calculada'] = df1.apply(completar_idade, axis=1)
df1

,anos_de_estudo,anos_de_experiencia,idade,salario,idade_calculada
0,18.0,6.0,35.0,16440.58,35.0
1,15.0,12.0,39.0,15992.51,39.0
2,18.0,11.0,43.0,15009.94,43.0
3,2.0,2.0,14.0,6964.13,14.0
4,NaN,6.0,18.0,7874.42,18.0
5,NaN,16.0,40.0,18329.56,40.0
6,NaN,4.0,34.0,15677.25,34.0
7,NaN,28.0,42.0,9429.71,42.0
8,NaN,27.0,53.0,15743.70,53.0
9,NaN,12.0,29.0,15922.81,29.0


In [348]:
df_filtered = df1[ (df1['anos_de_estudo'].notnull()) & (df1['anos_de_experiencia'].notnull()) & (df1['idade_calculada'].notnull()) ]
x = df_filtered[['anos_de_experiencia', 'idade_calculada']].to_numpy()
y = df_filtered['anos_de_estudo'].to_numpy()
model_anos_de_estudo = LinearRegression()
model_anos_de_estudo.fit(x, y)
score = model_anos_de_estudo.score(x, y)
score

0.975030483580227

In [349]:
df_filtered = df1[ (df1['anos_de_estudo'].notnull()) & (df1['anos_de_experiencia'].notnull()) & (df1['idade_calculada'].notnull()) ]
x = df_filtered[['anos_de_estudo', 'idade_calculada']].to_numpy()
y = df_filtered['anos_de_experiencia'].to_numpy()
model_anos_de_experiencia = LinearRegression()
model_anos_de_experiencia.fit(x, y)
score = model_anos_de_experiencia.score(x, y)
score

0.990240614585267

In [350]:
# minha função criada para completar o anos_de_estudo
def completar_anos_de_estudo(linha):
  if pd.isna(linha['anos_de_estudo']) and not pd.isna(linha['idade_calculada']):
    campos = linha[['anos_de_experiencia','idade_calculada']]
    return round(model_anos_de_estudo.predict([campos])[0])
  return linha['anos_de_estudo']

# apply: executar a função chamada 1x para cada linha
df1['anos_de_estudo_calculado'] = df1.apply(completar_anos_de_estudo, axis=1)
df1

,anos_de_estudo,anos_de_experiencia,idade,salario,idade_calculada,anos_de_estudo_calculado
0,18.0,6.0,35.0,16440.58,35.0,18.0
1,15.0,12.0,39.0,15992.51,39.0,15.0
2,18.0,11.0,43.0,15009.94,43.0,18.0
3,2.0,2.0,14.0,6964.13,14.0,2.0
4,NaN,6.0,18.0,7874.42,18.0,0.0
5,NaN,16.0,40.0,18329.56,40.0,12.0
6,NaN,4.0,34.0,15677.25,34.0,17.0
7,NaN,28.0,42.0,9429.71,42.0,3.0
8,NaN,27.0,53.0,15743.70,53.0,14.0
9,NaN,12.0,29.0,15922.81,29.0,5.0


In [351]:
# minha função criada para completar o anos_de_experiencia
def completar_anos_de_experiencia(linha):
  if pd.isna(linha['anos_de_experiencia']) and not pd.isna(linha['idade_calculada']):
    campos = linha[['anos_de_estudo','idade_calculada']]
    return round(model_anos_de_experiencia.predict([campos])[0])
  return linha['anos_de_experiencia']

# apply: executar a função chamada 1x para cada linha
df1['anos_de_experiencia_calculado'] = df1.apply(completar_anos_de_experiencia, axis=1)
df1

,anos_de_estudo,anos_de_experiencia,idade,salario,idade_calculada,anos_de_estudo_calculado,anos_de_experiencia_calculado
0,18.0,6.0,35.0,16440.58,35.0,18.0,6.0
1,15.0,12.0,39.0,15992.51,39.0,15.0,12.0
2,18.0,11.0,43.0,15009.94,43.0,18.0,11.0
3,2.0,2.0,14.0,6964.13,14.0,2.0,2.0
4,NaN,6.0,18.0,7874.42,18.0,0.0,6.0
5,NaN,16.0,40.0,18329.56,40.0,12.0,16.0
6,NaN,4.0,34.0,15677.25,34.0,17.0,4.0
7,NaN,28.0,42.0,9429.71,42.0,3.0,28.0
8,NaN,27.0,53.0,15743.70,53.0,14.0,27.0
9,NaN,12.0,29.0,15922.81,29.0,5.0,12.0


In [352]:
# Recalculo a minha regressão original utilizando as variáveis novas
x = df1[['idade_calculada',	'anos_de_estudo_calculado', 'anos_de_experiencia_calculado']]
y = df1['salario'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score_reconstruido = model.score(x, y)
print(score_reconstruido)

0.6544820749074767


In [353]:
print("O score original resultou em:", score_original, "e já o score reconstruído em:", score_reconstruido)

O score original resultou em: 0.9014024420114483 e já o score reconstruído em: 0.6544820749074767
